In [1]:
# Libraries
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import helperfunctions as hf
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2 
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.feature_selection import SelectKBest, f_regression, VarianceThreshold
from IPython.core.interactiveshell import InteractiveShell

# Notebook Settings 
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("error")
pd.set_option('display.max_columns', 500)
InteractiveShell.ast_node_interactivity = "all"
%load_ext autoreload
%autoreload 2

# Global variables
crop_seasons = list(range(1993,2017))
months_of_crop_season = list(range(4,12))
homogeneous_groups = list(range(1,5))

## Content
* [1. Read Data](#read_data)
* [2. Bias-Adjustment](#bias_adjustment)
* [3. Dataset Completion](#dataset_completion)
* [4. Modifications](#modification)
* [5. Climatology](#climatology)

## 1. Read Data <a name="read_data"></a>

Our approach requires three sources of climate data: seasonal climate models (hindcasts), observations, and climatology.
- **hindcasts**: There are three seasonal climate models that we requested data from: ECMWF, UKMO, NCEP + an unweighted average of their outputs to a multi-model ensemble (MME). Hindcasts are from 1993 to 2016 for four locations (zones) in Brazil. The locations were selected based on the findings from Nóia Júnior et al. ([2021](https://iopscience.iop.org/article/10.1088/1748-9326/ac26f3)). For each model, year, and location, we requested seven hindcasts, initialized at the beginning of each month during the wheat growing season from April to October and forecasting precipitation and temperature data until the end of the season.
- **observations**: Climate observations from the same four locations ([Nóia Júnior et al., 2021](https://iopscience.iop.org/article/10.1088/1748-9326/ac26f3)) from 1993 to 2016 along the wheat growing season from April to October were requested. This data is used for bias-adjustment of the hindcasts but also for the wheat yield forecast model. When a forecast is provided in month *m*, climate features from past month are supplemented with climate observations, while future months are based on forecasted climate features. Additionally, we need climate observations to calculate expected, *normal*, climate conditions (climatology) to benchmark our approach with. 
- **climatology**: For each location, month, climate variable, and year *y*, we compute the average from observations from the same location, month, climate variable, and all other years except year *y* from 1993 to 2016.

In [3]:
hindcasts = hf.read_raw_model_data() # daily; 1993-2016
observations = hf.read_observed_weather() # daily; 1993-2016
climatology = hf.create_climatology_data(observations) # monthly; Leave-One-Out 1993-2016

hindcasts.head(1)
observations.head(1)
climatology.head(1)

,,,,,time,tmean,tmax,tmin,rain
model,init_month,zone,year,month,,,,,
ECMWF,4,1,1993,4,1993-04-02,19.825406,26.905211,17.873243,5.589371


,,,,,time,tmean,tmax,tmin,rain
model,init_month,zone,year,month,,,,,
WS,11,1,1993,4,1993-04-01,21.9,26.6,17.2,0.0


,zone,year,month,tmean,tmax,tmin,rain
0,1,1993,4,19.311304,24.675797,13.946812,147.091304


## 2. Bias-Adjustment <a name="bias_adjustment"></a>

Biases are systematic errors between forecasts and observations that come from inaccuracies in the model design and the sensitivity of climate models to initial conditions (see, e.g. [ECMWF-Wiki](https://confluence.ecmwf.int/display/CKB/Seasonal+forecasts+and+the+Copernicus+Climate+Change+Service)). We use [scaled (normal) distribution mapping](https://hess.copernicus.org/articles/21/2649/2017/) to adjust biases in forecasted daily mean, maximum, and minimum air temperature. We do not apply any bias adjustment to rain forecasts as it did not lead to improvements in mean absolut error. We adjust temperature values by *model*, *init_month*, *zone*, and *month* for each year *y* using observations and hindcasts from all other years (Leave-One-Out) to avoid overfitting.

In [4]:
hindcasts_temp_adjusted = hf.adjust_temperature_bias(observations, hindcasts)

## 3. Dataset Completion <a name="dataset_completion"></a>

We need monthly climate features from April to October. Hindcasts provide data from the data of initialization until October. Data from the past needs to be supplemented with climate observations to complete the entire period. We validate that for each *model*, *init_month*, *zone*, and *year*, we have the same number of observations: \
30 days for April + 31 days for May + 30 days for June + 31 days for July + 31 days for Aug + 30 days for Sept + 31 days for Oct = 214 days.

In [6]:
hindcast_complete = hf.fill_missing_dates_with_observations(observations, hindcasts_temp_adjusted) 
hindcast_complete.reset_index().groupby(["model", "init_month", "zone", "year"]).size().unique()

# We concatenate the hindcasted daily values with the fully observed daily values.
climate_records_complete = pd.concat([hindcast_complete, observations]).sort_index()

array([214], dtype=int64)

**Remark:** We base our approach on the estimation model developped by Nóia Júnior et al. (2021). During the next chapter, we progressively modify the approach to work better with seasonal climate models. There are overall five experiments or modification steps that we conducted. These steps are concerned about the feature selection, design, or problem formulation.

## 4. Modifications <a name="modification"></a>

We will now read the national, and grouped detrended wheat yield data to be merged with our feature dataset. The wheat yield data was obtained from the [Brazilian Institute of Geography and Statistics](https://sidra.ibge.gov.br/tabela/1612). For more information on the data, see the other notebook *prepare_wheat_data*.

In [9]:
national_yield, grouped_yield = hf.read_wheat_yield_data()

# dataframe to store results of the following modifications
results_of_modifications = pd.DataFrame(0, index=pd.MultiIndex.from_product([list(range(1,6)), ["ECMWF", "NCEP", "UKMO", "MME"]]), columns=months_of_crop_season)

# models to validate the experiments
models = ["ECMWF", "NCEP", "UKMO", "MME"]

### 4.1 Exp. #1: Identical Approach (Nóia Júnior et al. 2021) with retrained weights

Here we use the same feature design, the same selected features for each group, the same months, and the same way of aggregation the group yield estimates to national yield.

Monthly features:
- number of days in a month above 32°C, or below 2°C.
- number of days in a months with rainfall above 0.1mm (rainy days) and 30mm (excessive rainfall)
- drought index; monthly main of daily mean, max, and minimum air temperature; monthly sum of daily rainfall

The reported weights from their paper were obtained by training on all years from 2001 to 2019. We have to be careful for data leakage and overfitting so we need to retrain the weights and use leave one out cross-validation to make our estimates. This way we obtain a more representative performance metric.

In [8]:
# calculate features including drought, heat stress, etc.
monthly_climate_features = hf.compute_monthly_climate_features(climate_records_complete)

# initial dataset with features and group yield
dataset_for_experiments = monthly_climate_features.reset_index().merge(grouped_yield, on=["zone", "year"], how="left")

We iterate over each month of initialization and model and obtain the forecasts that we can compare with the ground truth. The relative Root-Mean-Square Error is saved in the dataframe.

In [11]:
for im in months_of_crop_season:
    for model in models:
        res = hf.retrain_weights(dataset_for_experiments, national_yield, model=model, init=im)
        metric = 100 * mse(res["yield"], res["predicted"], squared=False)/(res["yield"].mean())
        # coefficient of determination'
        #metric = r2(res["yield"], res["predicted"])
        results_of_modifications.loc[(1, model), im] = np.round(metric, 2)
        
# display RMSE [%] performance
results_of_modifications.loc[1]

,4,5,6,7,8,9,10,11
ECMWF,26.70,25.31,29.32,29.76,28.92,20.69,10.92,10.1
NCEP,24.82,23.99,27.63,26.62,24.15,18.83,10.45,10.1
UKMO,23.76,25.06,27.55,28.24,28.18,17.94,11.14,10.1
MME,24.62,27.48,29.61,30.13,29.14,20.44,11.18,10.1


### 4.2 Exp. #2: Select new features

Nóia Júnior et al (2021) identified the best features for each agro-climatic homogeneous group using stepwise forward selection. Since we now work with a slightly different period of years, it can be beneficial to perform another feature selection step. Using cross-validation, features are selected based on correlation with the target. Correlated features (pearson > 0.9) are removed before.

In [12]:
relevant_columns = [c for c in dataset_for_experiments.columns if c not in ["model", "init_month", "zone", "year", "yield"]]
values = dataset_for_experiments.loc[(dataset_for_experiments["model"] == "WS"), relevant_columns]
cor_matrix = values.corr().abs().round(2)
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]
print(to_drop)

dataset_for_experiments_uncorrelated_features = dataset_for_experiments.drop(to_drop, axis=1)

['Tmax_April', 'Tmax_Aug', 'Tmax_July', 'Tmax_June', 'Tmax_May', 'Tmax_Oct', 'Tmax_Sep', 'Tmin_April', 'Tmin_Aug', 'Tmin_July', 'Tmin_June', 'Tmin_May', 'Tmin_Oct', 'Tmin_Sep', 'Hrainfall_Oct']


In [14]:
models = ["ECMWF", "NCEP", "UKMO", "MME"]
for im in months_of_crop_season:
    for model in models:
        # 8 features gave best overall performance
        res = hf.calculate_estimates(dataset_for_experiments_uncorrelated_features, national_yield, model=model, init=im, no_of_features=8)
        metric = 100 * mse(res["yield"], res["predicted"], squared=False)/(res["yield"].mean())
        # coefficient of determination
        #metric = r2(res["yield"], res["predicted"])
        results_of_modifications.loc[(2, model), im] = np.round(metric, 2)

results_of_modifications.loc[([1,2])]

4      5      6      7      8      9      10     11
1 ECMWF  26.70  25.31  29.32  29.76  28.92  20.69  10.92  10.10
  NCEP   24.82  23.99  27.63  26.62  24.15  18.83  10.45  10.10
  UKMO   23.76  25.06  27.55  28.24  28.18  17.94  11.14  10.10
  MME    24.62  27.48  29.61  30.13  29.14  20.44  11.18  10.10
2 ECMWF  12.52  14.65  14.17  13.52  12.74  12.42  10.98  10.11
  NCEP   14.07  14.84  15.62  13.00  12.81  12.72  11.71  10.11
  UKMO   13.15  14.57  14.11  13.09  12.56  12.68  11.41  10.11
  MME    12.64  14.74  15.03  13.97  13.19  12.60  11.53  10.11

We see significant RMSE improvement for forecasts issued from April to September for all types of climate models.

### 4.3 Exp. #3: Drop features with daily focus

Seasonal climate models are not capable to forecast, for example, the number of days in a month below 2°C. For more information about the capabilities, see for example the [forecast skill horizon](https://www.ecmwf.int/en/elibrary/8450-forecast-skill-horizon). We will drop all columns with such a daily focus and only use monthly aggregations of daily mean temperature and daily rainfall.

In [16]:
dataset_for_experiments_filtered_features = dataset_for_experiments_uncorrelated_features.loc[:, 
                                          ['model', 'init_month', 'zone', 'year', 
                                           'Tmean_April', 'Tmean_Aug', 'Tmean_July', 'Tmean_June', 'Tmean_May', 'Tmean_Oct', 'Tmean_Sep',
                                           'Rain_April', 'Rain_Aug', 'Rain_July', 'Rain_June', 'Rain_May', 'Rain_Oct', 'Rain_Sep', 
                                           'yield']
                                          ]

In [17]:
models = ["ECMWF", "NCEP", "UKMO", "MME"]
for im in months_of_crop_season:
    for model in models:
        res = hf.calculate_estimates(dataset_for_experiments_filtered_features, national_yield, model=model, init=im, no_of_features=8)
        metric = 100 * mse(res["yield"], res["predicted"], squared=False)/(res["yield"].mean())
        # coefficient of determination
        #metric = r2(res["yield"], res["predicted"])
        results_of_modifications.loc[(3, model), im] = np.round(metric, 2)

results_of_modifications.loc[([1, 2, 3])]

4      5      6      7      8      9      10     11
1 ECMWF  26.70  25.31  29.32  29.76  28.92  20.69  10.92  10.10
  NCEP   24.82  23.99  27.63  26.62  24.15  18.83  10.45  10.10
  UKMO   23.76  25.06  27.55  28.24  28.18  17.94  11.14  10.10
  MME    24.62  27.48  29.61  30.13  29.14  20.44  11.18  10.10
2 ECMWF  12.52  14.65  14.17  13.52  12.74  12.42  10.98  10.11
  NCEP   14.07  14.84  15.62  13.00  12.81  12.72  11.71  10.11
  UKMO   13.15  14.57  14.11  13.09  12.56  12.68  11.41  10.11
  MME    12.64  14.74  15.03  13.97  13.19  12.60  11.53  10.11
3 ECMWF  12.03  12.52  11.83  12.18  12.50  12.87  11.62  10.94
  NCEP   12.73  13.11  13.64  12.43  13.79  13.02  12.36  10.94
  UKMO   11.77  12.98  12.26  12.32  12.10  12.54  11.58  10.94
  MME    11.74  12.53  12.30  12.09  12.45  12.65  11.78  10.94

There are improvements especially for the early months during the crop season from April to July. In the second half of the season, performance did not change or became worse.

### 4.4 Exp. #4: Only use features from reproductive period

Previous studies suggested that climate conditions during the reproductive period of the crop cycle are most influencial for yield. For example, Iizumi et. al ([2018](https://www.sciencedirect.com/science/article/pii/S24058807173013469)), used climate data from the last three months of the crop season. We will follow their approach and only include climate features from August to October.

In [18]:
dataset_for_experiments_reproductive = dataset_for_experiments_filtered_features.loc[:, ['model', 'init_month', 'zone', 'year', 
                                                                                         'Tmean_Aug', 'Tmean_Oct', 'Tmean_Sep', 'Rain_Aug', 'Rain_Oct', 'Rain_Sep', 
                                                                                         'yield']]

In [19]:
models = ["ECMWF", "NCEP", "UKMO", "MME"]
for im in months_of_crop_season:
    for model in models:
        res = hf.calculate_estimates(dataset_for_experiments_reproductive, national_yield, model=model, init=im, no_of_features="all")
        metric = 100 * mse(res["yield"], res["predicted"], squared=False)/(res["yield"].mean())
        # coefficient of determination
        #metric = r2(res["yield"], res["predicted"])
        results_of_modifications.loc[(4, model), im] = np.round(metric, 2)
        
results_of_modifications.loc[([1, 2, 3, 4])]

4      5      6      7      8      9      10     11
1 ECMWF  26.70  25.31  29.32  29.76  28.92  20.69  10.92  10.10
  NCEP   24.82  23.99  27.63  26.62  24.15  18.83  10.45  10.10
  UKMO   23.76  25.06  27.55  28.24  28.18  17.94  11.14  10.10
  MME    24.62  27.48  29.61  30.13  29.14  20.44  11.18  10.10
2 ECMWF  12.52  14.65  14.17  13.52  12.74  12.42  10.98  10.11
  NCEP   14.07  14.84  15.62  13.00  12.81  12.72  11.71  10.11
  UKMO   13.15  14.57  14.11  13.09  12.56  12.68  11.41  10.11
  MME    12.64  14.74  15.03  13.97  13.19  12.60  11.53  10.11
3 ECMWF  12.03  12.52  11.83  12.18  12.50  12.87  11.62  10.94
  NCEP   12.73  13.11  13.64  12.43  13.79  13.02  12.36  10.94
  UKMO   11.77  12.98  12.26  12.32  12.10  12.54  11.58  10.94
  MME    11.74  12.53  12.30  12.09  12.45  12.65  11.78  10.94
4 ECMWF  11.67  11.89  11.21  11.49  11.15  11.18   9.50   8.83
  NCEP   13.75  12.66  14.09  12.55  12.50  11.74  10.06   8.83
  UKMO   11.74  12.49  11.16  12.23  11.06  11.00   9.35   8.83
  MME    12.04  12.03  11.82  11.83  11.14  11.07   9.49   8.83

There are significant improvements across the entire crop season, but especially for forecasts issued towards harvest.

### 4.5 Exp. #5: Forecast yield directly on national level

In the approach of [Nóia Júnior et al., 2021](https://iopscience.iop.org/article/10.1088/1748-9326/ac26f3), separate models were trained for each location (agro-climatic homogeneous groups) and their estimates where extrapolated to national level using harvested area estimates (average harvested area) for each group. We now choose a different approach, where we directly estimate national wheat yield and the model can decide which location and climate feature it can assign more importance to. We simply need to unstack the *zone* column. The feature names will now hold an additional suffix *n*, where *n* ranges from 1 to 4, indicating the location where that climate feature belongs to. 

#### Unstack features by zone

In [20]:
dataset_for_experiments_unstacked = dataset_for_experiments_reproductive.loc[:, [c for c in dataset_for_experiments_reproductive.columns if c != "yield"]].set_index(["zone", "model", "init_month", "year"]).unstack(0)
dataset_for_experiments_unstacked.columns = [str(s[0]) + "_" + str(s[1]) for s in dataset_for_experiments_unstacked.columns]
dataset_for_experiments_unstacked = dataset_for_experiments_unstacked.reset_index()
dataset_for_experiments_unstacked = dataset_for_experiments_unstacked.merge(national_yield, on="year", how="left") 
dataset_for_experiments_unstacked.head(2)

,model,init_month,year,Tmean_Aug_1,Tmean_Aug_2,Tmean_Aug_3,Tmean_Aug_4,Tmean_Oct_1,Tmean_Oct_2,Tmean_Oct_3,Tmean_Oct_4,Tmean_Sep_1,Tmean_Sep_2,Tmean_Sep_3,Tmean_Sep_4,Rain_Aug_1,Rain_Aug_2,Rain_Aug_3,Rain_Aug_4,Rain_Oct_1,Rain_Oct_2,Rain_Oct_3,Rain_Oct_4,Rain_Sep_1,Rain_Sep_2,Rain_Sep_3,Rain_Sep_4,yield
0,ECMWF,4,1993,15.056569,20.204620,18.552408,16.381428,19.002330,23.778185,22.431311,19.486069,16.206275,21.793791,20.112309,17.307033,165.319824,47.648926,87.138672,81.333008,223.339844,163.183594,199.042969,186.162109,177.917481,98.933105,144.587403,127.775879,2687.363636
1,ECMWF,4,1994,15.019860,19.965974,18.318693,15.976337,18.899465,23.598450,22.159399,19.119748,15.657911,21.280677,19.634187,16.936952,168.437500,49.804688,76.982422,76.577149,221.113281,149.296875,191.523438,160.224610,203.830566,115.625000,156.516113,141.743164,2737.764325


In [21]:
models = ["ECMWF", "NCEP", "UKMO", "MME"]
results = pd.DataFrame(0, index=models, columns=months_of_crop_season)
for im in months_of_crop_season:
    for model in models:
        res = hf.kfold_cross_validation(dataset_for_experiments_unstacked, model=model, init=im, no_of_features=8)
        # rmse
        metric = 100 * mse(res["yield"], res["predicted"], squared=False)/(res["yield"].mean())
        # coefficient of determination
        #metric = r2(res["yield"], res["predicted"])
        results_of_modifications.loc[(5, model), im] = np.round(metric, 2) 
        
results_of_modifications

4      5      6      7      8      9      10     11
1 ECMWF  26.70  25.31  29.32  29.76  28.92  20.69  10.92  10.10
  NCEP   24.82  23.99  27.63  26.62  24.15  18.83  10.45  10.10
  UKMO   23.76  25.06  27.55  28.24  28.18  17.94  11.14  10.10
  MME    24.62  27.48  29.61  30.13  29.14  20.44  11.18  10.10
2 ECMWF  12.52  14.65  14.17  13.52  12.74  12.42  10.98  10.11
  NCEP   14.07  14.84  15.62  13.00  12.81  12.72  11.71  10.11
  UKMO   13.15  14.57  14.11  13.09  12.56  12.68  11.41  10.11
  MME    12.64  14.74  15.03  13.97  13.19  12.60  11.53  10.11
3 ECMWF  12.03  12.52  11.83  12.18  12.50  12.87  11.62  10.94
  NCEP   12.73  13.11  13.64  12.43  13.79  13.02  12.36  10.94
  UKMO   11.77  12.98  12.26  12.32  12.10  12.54  11.58  10.94
  MME    11.74  12.53  12.30  12.09  12.45  12.65  11.78  10.94
4 ECMWF  11.67  11.89  11.21  11.49  11.15  11.18   9.50   8.83
  NCEP   13.75  12.66  14.09  12.55  12.50  11.74  10.06   8.83
  UKMO   11.74  12.49  11.16  12.23  11.06  11.00   9.35   8.83
  MME    12.04  12.03  11.82  11.83  11.14  11.07   9.49   8.83
5 ECMWF  11.56  11.90  11.23  11.44  10.58   9.49   7.88   6.01
  NCEP   14.34  14.88  14.70  12.74  11.84  10.45   9.26   6.01
  UKMO   12.28  12.71  10.79  12.13  11.07  10.61   7.64   6.01
  MME    12.12  12.47  11.76  11.62  10.47   9.86   7.90   6.01

#### Export results of each modification step

To be used for visualizations in **figures_for_manuscript**

In [22]:
results_of_modifications.to_csv("results_of_modification.csv")

## 5. Add Climatology <a name="climatology"></a>

We benchmark our final approach with climatology and need to include climatology features.

In [23]:
features_climatology = hf.create_climatology_features(dataset_for_experiments_reproductive, climatology)
features_complete = (pd
                     .concat([dataset_for_experiments_reproductive, features_climatology])
                     .sort_values(["model", "init_month", "zone", "year"])
                     .drop_duplicates()
                     .reset_index(drop=True))
features_complete = features_complete.loc[:, [c for c in features_complete.columns if c != "yield"]].set_index(["zone", "model", "init_month", "year"]).unstack(0)
features_complete.columns = [str(s[0]) + "_" + str(s[1]) for s in features_complete.columns]
features_complete = features_complete.reset_index()
features_complete = features_complete.merge(national_yield, on="year", how="left") 

### Summary of our dataset

Let's quickly summarize the data.
- There are 24 years, from 1993 to 2016
- For each year, we have 6 different model sources: ECMWF, NCEP, UKMO, MME, CLIMATE, WS (observations)
- WS has one data point per year, the other models have eight data points, one for each month of initialization from April to November
- This gives us 24 * (5 * 8 + 1) = 984 data points

In [24]:
features_complete.shape

(984, 28)

In [26]:
#features_complete.to_csv("kfold_cv_dataset.csv", index=False)

In [57]:
features_complete = pd.read_csv("kfold_cv_dataset.csv")

In [51]:
models = ["ECMWF", "NCEP", "UKMO", "MME", "CLIMATE", "WS"]
results = pd.DataFrame(0, index=models, columns=months_of_crop_season)
for im in months_of_crop_season:
    for model in models:
        res = hf.kfold_cross_validation(features_complete, model=model, init=im, no_of_features=8)
        # rmse
        metric = 100 * mse(res["yield"], res["predicted"], squared=False)/(res["yield"].mean())
        # coefficient of determination
        #metric = r2(res["yield"], res["predicted"])
        results.loc[model, im] = np.round(metric, 2)

results # coefficient of determination with 8 features

,4,5,6,7,8,9,10,11
ECMWF,11.56,11.90,11.23,11.44,10.58,9.49,7.88,6.01
NCEP,14.34,14.88,14.70,12.74,11.84,10.45,9.26,6.01
UKMO,12.28,12.71,10.79,12.13,11.07,10.61,7.64,6.01
MME,12.12,12.47,11.76,11.62,10.47,9.86,7.90,6.01
CLIMATE,11.75,11.75,11.75,11.75,11.75,10.27,9.13,6.01
WS,6.01,6.01,6.01,6.01,6.01,6.01,6.01,6.01
